In [2]:
from generate_maps import load_s3dis_point_cloud, get_topologial_map

file_path = 'data/noXYZ_area_5a_no_xyz/area_5a/3d/pointcloud.mat'  
point_sum_points, point_sum_colors, scene_names = load_s3dis_point_cloud(file_path)
print("success load!")


Keys in the .mat file: ['Area_5a']
Processing conferenceRoom_1...
Processing conferenceRoom_2...
Processing conferenceRoom_3...
Processing hallway_1...
Processing hallway_2...
Processing hallway_3...
Processing hallway_4...
Processing hallway_5...
Processing hallway_6...
Processing lobby_1...
Processing hallway_7...
Processing hallway_8...
Processing hallway_9...
Processing hallway_10...
Processing hallway_11...
Processing hallway_12...
Processing hallway_13...
Processing hallway_14...
Processing hallway_15...
Processing office_1...
Processing office_2...
Processing office_3...
Processing office_4...
Processing office_5...
Processing office_6...
Processing office_7...
Processing office_8...
Processing office_9...
Processing office_10...
Processing office_11...
Processing office_12...
Processing office_13...
Processing office_14...
Processing office_15...
Processing office_16...
Processing office_17...
Processing office_18...
Processing office_19...
Processing office_20...
Processing of

In [2]:
for i, name in enumerate(scene_names):
    if "lobby" in name:
        del point_sum_points[i]
        del point_sum_colors[i]
        del scene_names[i]

In [3]:
import generate_maps
import importlib
importlib.reload(generate_maps)

top_view_map, obstacle_map, separate_obstacle_map, all_centers = generate_maps.get_top_view_map(point_sum_points, point_sum_colors, scene_names, resolution=0.02)


In [13]:
from PIL import Image, ImageDraw, ImageFont
import random
import matplotlib.pyplot as plt
from io import BytesIO
import cv2
import matplotlib.pyplot as plt
import numpy as np

def generate_landmarks(s_start, s_end, top_view_map, obstacle_map):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(25, 25))
    ob_map = cv2.dilate((obstacle_map).astype('uint8'), kernel)

    pil_img = Image.fromarray(top_view_map)
    draw = ImageDraw.Draw(pil_img)

    font_size = 25
    # Load a font (make sure you specify a valid .ttf font path on your system)
    font = ImageFont.load_default(font_size)

    # Number of circles to draw
    num_labels = 80

    # Circle radius
    r = 20

    # Image dimensions
    height, width, _ = top_view_map.shape

    i = 1
    valid_points = {}
    while i < num_labels: 
        # Generate random (x, y) within image boundaries
        x = random.randint(r, width - r)
        y = random.randint(r, height - r)
        
        if ob_map[y, x] == 1:
            continue
        
        # Check distance from all previously placed points
        too_close = False
        min_dist = 100
        for label_key, (px, py) in valid_points.items():
            if (y - px)**2 + (x - py)**2 < min_dist**2:
                too_close = True
                break
        
        # If this point is too close to an existing point, skip it
        if too_close:
            continue
        
        # Draw the circle
        draw.ellipse(
            [(x - r, y - r), (x + r, y + r)],
            fill="white",       # circle fill color
            outline="black",    # circle outline color
            width=2             # outline thickness
        )
        
        # Center the text inside the circle
        label = str(i)
        text_x = x - (font_size / 2)
        text_y = y - (font_size / 2)
        draw.text((text_x, text_y), label, fill="black", font=font)
        
        valid_points[str(i)] = (y, x)
        i += 1
        
        
    # mark start and end
    font_size = 30
    # Load a font (make sure you specify a valid .ttf font path on your system)
    font = ImageFont.load_default(font_size)
    x = s_start[1]
    y = s_start[0]
    # Draw the circle
    draw.ellipse(
        [(x - r, y - r), (x + r, y + r)],
        fill="white",       # circle fill color
        outline="red",    # circle outline color
        width=4             # outline thickness
    )

    # Center the text inside the circle
    label = 'S'
    text_x = x - (font_size / 3)
    text_y = y - (font_size / 1.5)
    draw.text((text_x, text_y), label, fill="red", font=font)
    valid_points[label] = (y, x)


    x = s_end[1]
    y = s_end[0]
    # Draw the circle
    draw.ellipse(
        [(x - r, y - r), (x + r, y + r)],
        fill="white",       # circle fill color
        outline="red",    # circle outline color
        width=4             # outline thickness
    )

    # Center the text inside the circle
    label = 'D'
    text_x = x - (font_size / 3)
    text_y = y - (font_size / 1.5)
    draw.text((text_x, text_y), label, fill="red", font=font)
    valid_points[label] = (y, x)

        
    pil_img.show()
    buffered = BytesIO()
    pil_img.save(buffered, format="JPEG")

    return buffered, valid_points

In [10]:
import networkx as nx
import cv2
import numpy as np
from io import BytesIO
import path_finding
import system_prompt
import chat_utils

importlib.reload(path_finding)
importlib.reload(chat_utils)

import os

filename = "5a"


def get_all_candidate_maps(source_node, target_node):

    s_start = all_centers[source_node]
    s_end = all_centers[target_node]
    
    buffered, valid_points = generate_landmarks(s_start, s_end, top_view_map, obstacle_map)
    obstacle_map_rgb = np.stack([obstacle_map] * 3, axis=-1)
    # obstacle_map_rgb = (obstacle_map_rgb * 255).astype(np.uint8)
    # obs_with_lm = path_finding.write_landmark_name(obstacle_map_rgb, valid_points)
    # # obs_with_lm = Image.fromarray(obstacle_map_rgb)
    # buffered_obs = BytesIO()
    # obs_with_lm.save(buffered_obs, format="JPEG")
    
    image_path = "/home/ybg/Project/Privacy-Aware-Nav/img/4_example.PNG"
    message = chat_utils.path_message_prepare(system_prompt.find_path_prompt, buffered, image_path)
    response_message = chat_utils.chat_with_gpt4v(message)
    ground_json = response_message["paths"]
    
    all_topo_paths = []
    for k, v in ground_json.items():
        all_topo_paths.append(v)
        
    all_geo_paths = path_finding.path_plan_from_topo_graph_for_large(all_topo_paths, obstacle_map, valid_points)

    save_directory = f"map_with_path/area_{filename}/{source_node}-{target_node}/"
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
        
    candidate_map_list = []
    for i, geo_path in enumerate(all_geo_paths):
        map_with_path = top_view_map.copy()
        x_indices, y_indices = zip(*geo_path)
        path_map = np.zeros(obstacle_map.shape)
        path_map[x_indices, y_indices] = 1
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5, 5))
        path_map = cv2.dilate((path_map).astype('uint8'), kernel)
        map_with_path[path_map == 1] = [255, 0 , 0]
        
        map_with_path = path_finding.write_number(map_with_path, i)
        buffered = BytesIO()
        map_with_path.save(buffered, format="JPEG")
        candidate_map_list.append(buffered)

        # 使用 Matplotlib 显示图像
        # 使用 Matplotlib 显示图像.
        # map_with_path.save(save_directory+f"vis_{i}.png")
        plt.imshow(map_with_path)
        plt.axis('off')  # 隐藏坐标轴
        plt.show()
        
        
    # self-critique
    critique_message = chat_utils.critique_message_prepare(system_prompt.self_critique_prompt, candidate_map_list)

    response_message = chat_utils.chat_with_gpt4v(message+critique_message)
    ground_json = response_message["self_critique"]
    print(ground_json)
    while ground_json == "0":
        all_topo_paths = []
        for k, v in response_message["paths"].items():
            all_topo_paths.append(v)
            
        all_geo_paths = path_finding.path_plan_from_topo_graph_for_large(all_topo_paths, obstacle_map, valid_points)

        candidate_map_list = []
        for i, geo_path in enumerate(all_geo_paths):
            map_with_path = top_view_map.copy()
            x_indices, y_indices = zip(*geo_path)
            path_map = np.zeros(obstacle_map.shape)
            path_map[x_indices, y_indices] = 1
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5, 5))
            path_map = cv2.dilate((path_map).astype('uint8'), kernel)
            map_with_path[path_map == 1] = [255, 0 , 0]
            
            map_with_path = path_finding.write_number(map_with_path, i)
            # map_with_path = path_finding.write_scene_name(map_with_path, valid_points)
            buffered = BytesIO()
            map_with_path.save(buffered, format="JPEG")
            candidate_map_list.append(buffered)

            # 使用 Matplotlib 显示图像
            # 使用 Matplotlib 显示图像.
            # map_with_path.save(save_directory+f"vis_{i}.png")
            plt.imshow(map_with_path)
            plt.axis('off')  # 隐藏坐标轴
            plt.show()
            
        critique_message = chat_utils.critique_message_prepare(system_prompt.self_critique_prompt, candidate_map_list)

        response_message = chat_utils.chat_with_gpt4v(message+critique_message)
        ground_json = response_message["self_critique"]
        
    return candidate_map_list, all_geo_paths

In [6]:
importlib.reload(path_finding)
from PIL import Image
from astar import AStar

def get_metric(source_node, target_node, all_geo_paths):
    
    evaluation_map = path_finding.get_evaulation_map(separate_obstacle_map, source_node, target_node, scene_names, 15)

    office_obstacle_map_array = np.array(separate_obstacle_map)
    office_obstacle_map = np.max(office_obstacle_map_array, axis=0)

    plt.imshow(evaluation_map)
    plt.axis('off')  # 隐藏坐标轴
    plt.show()

    s_start = all_centers[source_node]
    s_goal = all_centers[target_node]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10, 10))
    ob_map = cv2.dilate((obstacle_map).astype('uint8'), kernel)
    astar = AStar(s_start, s_goal, "manhattan", ob_map)
    path, visited = astar.searching()
    x_indices, y_indices = zip(*path)
    a_star_privacy = sum(evaluation_map[x_indices, y_indices])
    a_star_distance = len(path)
    
    privacy_value_list = []
    distance_list = []
    for i, geo_path in enumerate(all_geo_paths):
        x_indices, y_indices = zip(*geo_path)
        # print(gaussian_modulation[x_indices[0], y_indices[0]])
        print("value: ", sum(evaluation_map[x_indices, y_indices]))
        print("distance: ", len(geo_path))
        privacy_value_list.append(sum(evaluation_map[x_indices, y_indices]))
        distance_list.append(len(geo_path))
        
    return privacy_value_list, distance_list, a_star_privacy, a_star_distance

In [14]:
import chat_utils
import ast
import concurrent.futures
from collections import Counter
import system_prompt
importlib.reload(chat_utils)
importlib.reload(system_prompt)
importlib.reload(path_finding)

num_samples = 5
navigation_instruct1 = "send a classified file from an office to another office"
navigation_instruct2 = "send a fragile equipment from an office to a conference room"
navigation_instruct3 = "send a private medicine from an office to the bathroom"

source_nodes = ["office_3", "office_22", "office_39"]
target_nodes = ["office_13", "conferenceRoom_2", "WC_2"]
# source_nodes = ["office_39"]
# target_nodes = ["WC_2"]

total_shortest_score = 0
total_shortest_distance = 0
total_privacy_score = 0
total_privacy_distance = 0
for source_node, target_node in zip(source_nodes, target_nodes):
    candidate_map_list, all_geo_paths = get_all_candidate_maps(source_node, target_node)
    p_v, dis, a_star_privacy, a_star_distance = get_metric(source_node, target_node, all_geo_paths)
    
    total_shortest_score += a_star_privacy
    total_shortest_distance += a_star_distance
    
    navigation_instruct = navigation_instruct1
    if "conference" in target_node:
        navigation_instruct = navigation_instruct2
    elif "WC" in target_node:
        navigation_instruct = navigation_instruct3
    message = chat_utils.message_prepare(system_prompt.infer_prompt, candidate_map_list, navigation_instruct)
            
    ground_json = chat_utils.chat_with_gpt4v(message)
    print(ground_json["path_id"])
    index = int(ground_json["path_id"])

    print("select ", index)
    total_privacy_score += p_v[index]
    total_privacy_distance += dis[index]
    
    
print("A* baseline result: ")
print("total_shortest_score: ", total_shortest_score)
print("total_shortest_distance: ", total_shortest_distance)

print("PAN baseline result: ")
print("total_privacy_score: ", total_privacy_score)
print("total_privacy_distance: ", total_privacy_distance)

KeyboardInterrupt: 